In [249]:
import os
import sys
#import urllib.request
import pandas as pd
import glob
import csv

In [254]:
# January data
files

['20170218.txt', '2017024.txt', '20170211.txt', '20170225.txt']

In [255]:
files = glob.glob('*20170*.txt')   


jdf = pd.concat([pd.read_csv(file) for file in files],axis=0 ,ignore_index=True)

In [240]:
jdf .shape

(400965, 11)

### Code to calculate the number of daylight hours and set the daylight saving time entry hours to original time to have consistency in the dataset

In [256]:
import datetime
from datetime import timedelta
#from datetime import datetime.strptime
import pytz
def getUTCtransitiontimes():
    df = pd.DataFrame(columns=['YEAR' ,'DTS_START' , 'DTS_END'])
    tz = pytz.timezone("America/New_York")
    dfutt_tmp = pd.DataFrame(tz._utc_transition_times , columns=["DATETIME"])
    dfutt_tmp['YEAR'] = dfutt_tmp['DATETIME'].map(lambda x: x.date().year)
    
    for name,group in dfutt_tmp.groupby(dfutt_tmp['YEAR']):
        if len(group) > 1:
            s = [{'YEAR': name , 'DTS_START' : group['DATETIME'].values[0] ,'DTS_END' : group['DATETIME'].values[1] }]
            df = df.append(s)
            
    df = df.set_index(['YEAR'])
            
    return df
        

    

    

In [257]:
dfUTCtt = getUTCtransitiontimes()

In [260]:
dfUTCtt.dtypes

DTS_START    datetime64[ns]
DTS_END      datetime64[ns]
dtype: object

In [ ]:
import pandas as pd

In [269]:
dfUTCtt['DTS_END'] = pd.to_datetime(dfUTCtt['DTS_END'])

In [273]:
dfUTCtt['DAYLIGHT_HOURS'] = dfUTCtt['DTS_END'].dt.hour +12  - dfUTCtt['DTS_START'].dt.hour

In [275]:
dfUTCtt.tail()

,DTS_START,DTS_END,DAYLIGHT_HOURS
YEAR,,,
2033,2033-03-13 07:00:00,2033-11-06 06:00:00,11
2034,2034-03-12 07:00:00,2034-11-05 06:00:00,11
2035,2035-03-11 07:00:00,2035-11-04 06:00:00,11
2036,2036-03-09 07:00:00,2036-11-02 06:00:00,11
2037,2037-03-08 07:00:00,2037-11-01 06:00:00,11


## Keep only the regular entries . Discard the outliers and audit entries

In [242]:

dfUTCtt = getUTCtransitiontimes()
    
jdf = jdf[jdf['DESC'] == 'REGULAR']
    
    #adjust for daylight saving time (dst)
jdf['DATETIME'] = jdf['DATE'] + " " + jdf['TIME']
    
jdf['DATETIME'] = jdf['DATETIME'].map(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y %H:%M:%S"))
    


In [ ]:
dfUTCtt.head()

In [243]:
jdf.ix[jdf['DATETIME'].map(lambda x: ((x > dfUTCtt.loc[x.date().year].values[0]) & (x < dfUTCtt.loc[x.date().year].values[1]))),
           'DATETIME'] = jdf['DATETIME'].map(lambda x: x - timedelta(hours=1))

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/ADS/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [244]:

    
    
jdf.ix[(jdf['DATETIME'].map(lambda x: x.hour in [0,4,8,12,16,20,24])) ,'DATETIME'] = jdf['DATETIME'].map(lambda x: x - timedelta(hours=1))
    
jdf['DATE'] = jdf['DATETIME'].map(lambda x:x.date())
jdf['TIME'] = jdf['DATETIME'].map(lambda x:x.time())
    
    
    #remove odd reporting cycles
    

    
    
    

    

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/ADS/lib/python3.5/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [245]:
jdf = jdf[ ( jdf['TIME'] == datetime.datetime.strptime("03:00:00" , "%H:%M:%S").time())
        | ( jdf['TIME'] == datetime.datetime.strptime("07:00:00" , "%H:%M:%S").time())
        | ( jdf['TIME'] == datetime.datetime.strptime("11:00:00" , "%H:%M:%S").time())
        | ( jdf['TIME'] == datetime.datetime.strptime("15:00:00" , "%H:%M:%S").time())
        | ( jdf['TIME'] == datetime.datetime.strptime("19:00:00" , "%H:%M:%S").time())
        | ( jdf['TIME'] == datetime.datetime.strptime("23:00:00" , "%H:%M:%S").time())]
    
    
jdf = jdf[jdf.ENTRIES > 0]
    

In [246]:
jdf.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
400960,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2018-12-07,03:00:00,REGULAR,5554,366,2018-12-07 03:00:00
400961,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2018-12-07,07:00:00,REGULAR,5554,366,2018-12-07 07:00:00
400962,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2018-12-07,11:00:00,REGULAR,5554,366,2018-12-07 11:00:00
400963,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2018-12-07,15:00:00,REGULAR,5554,366,2018-12-07 15:00:00
400964,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2018-12-07,19:00:00,REGULAR,5554,366,2018-12-07 19:00:00


In [247]:
jdf.to_csv('./t18urstileDec18.csv')